# Training RNNT Whisper Small VI with Multi-GPU Accelerate

Notebook này dùng để huấn luyện mô hình StreamingRNNT cho tiếng Việt với hỗ trợ multi-GPU thông qua thư viện Accelerate từ Hugging Face.


## 1. Cài đặt môi trường


In [ ]:
# Cài đặt các gói phụ thuộc
!pip install accelerate sentencepiece jiwer librosa tqdm loguru tensorboard

## 2. Tải mã nguồn


In [ ]:
# Clone repository từ GitHub
!git clone https://github.com/nguyendevwk/rnnt_whisper_small_vi.git
%cd rnnt_whisper_small_vi

## 3. Thêm script cho multi-GPU training


In [ ]:
%%readfile train_accelerate.py
# Nội dung file train_accelerate.py sẽ được tự động thêm vào đây

In [ ]:
%%writefile train_accelerate.py
# Nội dung file train_accelerate.py sẽ được tự động thêm vào đây

In [ ]:
%%writefile run_accelerate.py
# Nội dung file run_accelerate.py sẽ được tự động thêm vào đây

## 4. Tạo cấu hình Accelerate


In [ ]:
# Kiểm tra GPU
!nvidia-smi

In [ ]:
# Tạo cấu hình Accelerate cho multi-GPU training
%%writefile accelerate_config.yaml
compute_environment: LOCAL_MACHINE
distributed_type: MULTI_GPU
downcast_bf16: 'no'
gpu_ids: all
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 0  # 0 sẽ tự động sử dụng tất cả GPU có sẵn
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

## 5. Chuẩn bị dữ liệu

Đoạn này giả định rằng bạn đã có dữ liệu sẵn, với các file manifest ở định dạng JSONL. File manifest có cấu trúc JSON với các trường bắt buộc: `audio_filepath`, `duration`, `text` và `offset`.


In [ ]:
# Kiểm tra đường dẫn manifest
TRAIN_MANIFEST = "/kaggle/working/data/train/train_data.jsonl"
VAL_MANIFEST = "/kaggle/working/data/test/test_data.jsonl"

import os

# Tạo thư mục nếu cần
os.makedirs(os.path.dirname(TRAIN_MANIFEST), exist_ok=True)
os.makedirs(os.path.dirname(VAL_MANIFEST), exist_ok=True)

# Kiểm tra xem file manifest có tồn tại không
if os.path.exists(TRAIN_MANIFEST):
    print(f"Train manifest đã tồn tại: {TRAIN_MANIFEST}")
    # Đọc vài dòng đầu tiên
    !head -n 2 {TRAIN_MANIFEST}
else:
    print(f"Train manifest không tồn tại: {TRAIN_MANIFEST}")
    print("Bạn cần chuẩn bị file manifest trước khi huấn luyện.")

if os.path.exists(VAL_MANIFEST):
    print(f"\nVal manifest đã tồn tại: {VAL_MANIFEST}")
    # Đọc vài dòng đầu tiên
    !head -n 2 {VAL_MANIFEST}
else:
    print(f"\nVal manifest không tồn tại: {VAL_MANIFEST}")
    print("Bạn cần chuẩn bị file manifest trước khi huấn luyện.")

# Kiểm tra thư mục đầu ra
OUTPUT_DIR = "./checkpoints"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"\nThư mục đầu ra: {OUTPUT_DIR}")

## 6. Chạy huấn luyện với multi-GPU


In [ ]:
# Tạo thư mục để lưu checkpoint
!mkdir -p ./checkpoints
!mkdir -p /kaggle/working/checkpoints

In [ ]:
# Chạy huấn luyện với run_accelerate.py
!python run_accelerate.py \
    --batch_size 32 \
    --max_epochs 50 \
    --num_workers 4 \
    --output_dir "./checkpoints" \
    --precision "bf16-mixed" \
    --lr 1e-4 \
    --train_manifest "{TRAIN_MANIFEST}" \
    --val_manifest "{VAL_MANIFEST}" \
    --base_path "/kaggle/working/" \
    --tokenizer_model_path "./weights/tokenizer_spe_bpe_v1024_pad/tokenizer.model" \
    --augment

## 7. Theo dõi huấn luyện với TensorBoard


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./checkpoints/logs

## 8. Tiếp tục huấn luyện từ checkpoint


In [ ]:
# Tiếp tục huấn luyện từ checkpoint đã lưu
!python run_accelerate.py \
    --batch_size 32 \
    --max_epochs 50 \
    --num_workers 4 \
    --output_dir "./checkpoints" \
    --precision "bf16-mixed" \
    --lr 5e-5 \
    --train_manifest "{TRAIN_MANIFEST}" \
    --val_manifest "{VAL_MANIFEST}" \
    --base_path "/kaggle/working/" \
    --tokenizer_model_path "./weights/tokenizer_spe_bpe_v1024_pad/tokenizer.model" \
    --resume_from_checkpoint "./checkpoints/rnnt-latest.pt" \
    --augment

## 9. Kiểm tra checkpoint đã lưu


In [ ]:
# Liệt kê các checkpoint đã lưu
!ls -la ./checkpoints/

# Đọc thông tin cấu hình
import json
try:
    with open('./checkpoints/training_config.json', 'r') as f:
        config = json.load(f)
    print("\nCấu hình huấn luyện:")
    for key, value in config.items():
        print(f"{key}: {value}")
except Exception as e:
    print(f"Không thể đọc file cấu hình: {e}")

## 10. Lưu ý quan trọng

1. **Manifest**: Phải đảm bảo rằng đường dẫn trong manifest file chính xác
2. **GPU**: Để tận dụng multi-GPU, hãy đảm bảo môi trường Kaggle của bạn có 2 GPU trở lên
3. **Checkpoints**: Kaggle notebooks chỉ giữ lại các file trong thư mục `/kaggle/working/`, nên bạn cần đảm bảo rằng checkpoints được lưu vào đó
4. **Xem kết quả**: Sử dụng TensorBoard để theo dõi quá trình huấn luyện
